In [4]:
pip install selenium pandas 

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
import time

In [3]:
# Initialize the WebDriver
driver = webdriver.Chrome()

In [5]:
try:
    # IMDb 2024 Movies URL
    url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=news"
    driver.get(url)
    driver.maximize_window()
    time.sleep(2)
    print(driver.title)
    
    while True:
        try:
            # Locate the "Read More" button
            element = driver.find_element(
                By.XPATH,
                '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button'
            )

            # Scroll the button into view
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
            time.sleep(1)

            # Click the button
            element.click()
            print("Clicked 'Read More' button.")
            time.sleep(1)  # Wait for new content to load

        except NoSuchElementException:
            print("No 'Read More' button found. All data loaded.")
            break

        except ElementClickInterceptedException:
            print("Button is blocked by another element. Retrying...")
            time.sleep(2)

        except TimeoutException:
            print("Operation timed out. Retrying...")
            time.sleep(2)

        except Exception as e:
            print(f"Unexpected error: {e}")
            break

finally:
    print("Successfully retrieved all the data.")


Movie, News, Release date between 2024-01-01 and 2024-12-31 (Sorted by Popularity Ascending)
No 'Read More' button found. All data loaded.
Successfully retrieved all the data.


In [21]:
try:
    # Locate all movie items
    movies = driver.find_elements(By.XPATH,'//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li')
    # Initialize a dictionary to store data by genre
    genre_data = {}
    
    # Extract details for each movie
    for movie in movies:
        try:
            # Movie Name
            name = movie.find_element(By.CSS_SELECTOR,'h3[class="ipc-title__text"]').text.split(". ", 1)[1]

            # Genre
            genre = movie.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[1]/div/div/div[2]/button[3]/span').text.strip()

            # Ratings
            try:
                rating = movie.find_element(By.CSS_SELECTOR, "span[class='ipc-rating-star--rating']").text.strip()
            except:
                rating = "N/A"

            # Voting Counts
            try:
                votes = movie.find_element(By.CSS_SELECTOR, "span[class='ipc-rating-star--voteCount']").text.replace("(", "").replace(")", "")
            except:
                votes = "N/A"

            # Duration
            try:
                duration = movie.find_element(By.XPATH, './div/div/div/div[1]/div[2]/div[2]/span[2]').text
            except:
                duration = "N/A"

            # Split genres and store data in a dictionary
            for g in genre.split(", "):
                if g not in genre_data:
                    genre_data[g] = []
                genre_data[g].append({"Movie Name": name, "Rating": rating, "Votes": votes, "Duration": duration, "Genre": genre})
                print(genre_data)
        except Exception as e:
            print(f"Error processing movie: {e}")
except Exception as e:
    print(f"Error retrieving movie list: {e}")
    

{'News': [{'Movie Name': 'The Fall of 2008', 'Rating': 'N/A', 'Votes': 'N/A', 'Duration': '1h 1m', 'Genre': 'News'}]}
{'News': [{'Movie Name': 'The Fall of 2008', 'Rating': 'N/A', 'Votes': 'N/A', 'Duration': '1h 1m', 'Genre': 'News'}, {'Movie Name': 'Every Christopher Nolan Movie Ranked', 'Rating': '8.6', 'Votes': ' 17', 'Duration': '59m', 'Genre': 'News'}]}
{'News': [{'Movie Name': 'The Fall of 2008', 'Rating': 'N/A', 'Votes': 'N/A', 'Duration': '1h 1m', 'Genre': 'News'}, {'Movie Name': 'Every Christopher Nolan Movie Ranked', 'Rating': '8.6', 'Votes': ' 17', 'Duration': '59m', 'Genre': 'News'}, {'Movie Name': 'The Turning Point: XCLD: The Story of Cancel Culture', 'Rating': '5.3', 'Votes': ' 30', 'Duration': '44m', 'Genre': 'News'}]}
{'News': [{'Movie Name': 'The Fall of 2008', 'Rating': 'N/A', 'Votes': 'N/A', 'Duration': '1h 1m', 'Genre': 'News'}, {'Movie Name': 'Every Christopher Nolan Movie Ranked', 'Rating': '8.6', 'Votes': ' 17', 'Duration': '59m', 'Genre': 'News'}, {'Movie Name'

In [232]:
# Close the WebDriver
driver.quit()

In [23]:
# Save data to CSV files
output_dir = "IMDB_2024_Genres"
os.makedirs(output_dir, exist_ok=True)

for genre, movies in genre_data.items():
    df = pd.DataFrame(movies)
    file_name = os.path.join(output_dir, f"{genre}.csv")
    df.to_csv(file_name, index=False)
    print(f"Saved data for genre '{genre}' to '{file_name}'")
   

Saved data for genre 'News' to 'IMDB_2024_Genres\News.csv'
